<a href="https://colab.research.google.com/github/gabrieladamasceno/Attacks_5G/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Acessar dataset
drive.mount('/content/gdrive')

# Carregar o dataset
encoded = pd.read_csv("/content/gdrive/MyDrive/Datasets/Attacks/Encoded/Encoded.csv")

Mounted at /content/gdrive


In [10]:
'''
# Verificar valores nulos
print(encoded.isnull().sum())

# Verificar valores infinitos
print(np.isinf(encoded).sum())

# Selecionar apenas colunas numéricas
numeric_cols = encoded.select_dtypes(include=[np.number]).columns.tolist()

# Verificar valores infinitos apenas nas colunas numéricas
infinite_counts = np.isinf(encoded[numeric_cols]).sum()
print(infinite_counts)

encoded.fillna(0, inplace=True)

# Verificar se ainda há valores NaN
nan_columns = encoded.isna().any()
print("Colunas com valores NaN ainda:", encoded.columns[nan_columns].tolist())
'''

Unnamed: 0    0
Seq           0
Dur           0
RunTime       0
Mean          0
             ..
cs4           0
cs6           0
cs7           0
ef            0
nan           0
Length: 96, dtype: int64


In [21]:
# Usando shape
num_amostras = encoded.shape[0]
print(f"Número de amostras: {num_amostras}")

# Usando len()
num_amostras_len = len(encoded)
print(f"Número de amostras (usando len): {num_amostras_len}")

Número de amostras: 1215890
Número de amostras (usando len): 1215890


In [24]:
# Exibir os nomes de todas as colunas
colunas = encoded.columns.tolist()  # Converte o Index em uma lista
print("Nomes das colunas:")
for coluna in colunas:
    print(coluna)

Nomes das colunas:
Unnamed: 0
Seq
Dur
RunTime
Mean
Sum
Min
Max
sTos
dTos
sTtl
dTtl
sHops
dHops
TotPkts
SrcPkts
DstPkts
TotBytes
SrcBytes
DstBytes
Offset
sMeanPktSz
dMeanPktSz
Load
SrcLoad
DstLoad
Loss
SrcLoss
DstLoss
pLoss
SrcGap
DstGap
Rate
SrcRate
DstRate
SrcWin
DstWin
sVid
dVid
SrcTCPBase
DstTCPBase
TcpRtt
SynAck
AckDat
Label
Attack Type
Attack Tool
 *        
 *    V   
 *    f   
 e        
 e    f   
 e &      
 e *      
 e d      
 e g      
 e i      
 e r      
 e s      
 eU       
e        
arp
icmp
ipv6-icmp
llc
lldp
sctp
tcp
udp
ACC
CON
ECO
FIN
INT
NRS
REQ
RSP
RST
TST
URP
Shutdown
Start
Status
39
4
52
54
af11
af12
af41
cs0
cs4
cs6
cs7
ef
nan


In [25]:
# Verificar o tipo da coluna 'Label'
tipo_coluna = encoded['e        '].dtype
print(f"O tipo da coluna 'e' é: {tipo_coluna}")

O tipo da coluna 'e' é: float64


In [26]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.optimizers import Adam


# Definir as features que você deseja usar
features = [
    'Seq', 'Offset', 'sTtl', 'e        ', 'tcp', 'AckDat', 'RST', 'INT', 'TcpRtt', 'icmp',
    'sMeanPktSz', 'FIN', 'sHops', 'Mean', 'dTtl', 'SrcBytes', 'TotBytes',
    'dMeanPktSz', 'Status', 'cs0', 'SrcWin', 'sTos', 'CON', 'REQ', 'Loss'
]

# Separar as features e os rótulos
X = encoded[features]  # Usar apenas as colunas selecionadas
y = encoded['Label'].values

# Codificar os rótulos para valores numéricos
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Transforma 'Benign' e 'Malign' em 0 e 1

# Identificar colunas categóricas (se houver)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Aplicar OneHotEncoder nas colunas categóricas
transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)  # Codificar colunas categóricas
    ],
    remainder='passthrough'  # Manter as colunas numéricas
)

X = transformer.fit_transform(X)  # Transformar as features

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [27]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir a estrutura da MLP
model = Sequential([
    Dense(96, activation='relu', input_shape=(X.shape[1],)),  # Camada de entrada com 96 neurônios
    Dropout(0.5),  # Adicionando Dropout
    Dense(64, activation='relu'),  # Primeira camada oculta
    Dropout(0.5),  # Adicionando Dropout
    Dense(64, activation='relu'),  # Segunda camada oculta
    Dropout(0.5),  # Adicionando Dropout
    Dense(64, activation='relu'),  # Terceira camada oculta
    Dropout(0.5),  # Adicionando Dropout
    Dense(1, activation='sigmoid')  # Camada de saída para classificação binária
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=5, batch_size=100, validation_data=(X_test, y_test))

Epoch 1/5
9728/9728 ━━━━━━━━━━━━━━━━━━━━ 44s 4ms/step - accuracy: 0.3917 - loss: nan - val_accuracy: 0.3939 - val_loss: nan
Epoch 2/5
9728/9728 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.3926 - loss: nan - val_accuracy: 0.3939 - val_loss: nan
Epoch 3/5
9728/9728 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step - accuracy: 0.3937 - loss: nan - val_accuracy: 0.3939 - val_loss: nan
Epoch 4/5
9728/9728 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - accuracy: 0.3924 - loss: nan - val_accuracy: 0.3939 - val_loss: nan
Epoch 5/5
9728/9728 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - accuracy: 0.3929 - loss: nan - val_accuracy: 0.3939 - val_loss: nan
